In [ ]:
import pandas as pd
import ast

In [ ]:
def str_to_list(x):
    try:
        return list(ast.literal_eval(x))
    except:
        return None


def string_to_list_dataframe(df):
    columns = df.columns.tolist()
    columns_w_lists = []
    for column in columns:
        if df[column].astype(str). \
                apply(lambda x: x.startswith('[') and x.endswith(']')) \
                .astype(int).mean() > 0.8:
            columns_w_lists.append(column)
    for column in columns_w_lists:
        df[column] = df[column].apply(lambda x: str_to_list(x))
        df = df[~df[column].isna()]
    return df

# Load data

In [ ]:
df = pd.read_csv('processed_data.csv')

In [ ]:
df = string_to_list_dataframe(df)

# See number of unique tokens

In [ ]:
from itertools import chain

In [ ]:
len(set(chain(*list(chain(*df['review_translate_sentences_lemma'].values.tolist())))))

In [ ]:
len(set(chain(*list(chain(*df['review_translate_sentences_tokens'].values.tolist())))))

# Train BPE tokenizer

In [ ]:
!ls /root

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, Regex
import tokenizers
from nltk.tokenize import regexp_tokenize

In [ ]:
df['review_translate'] = df['review_translate'].str.lower()

In [ ]:
pre_tok = pre_tokenizers.Split(Regex(r"[\w'-]+|[^\w\s'-]+"),'removed', True)

In [ ]:
bpe = models.BPE(min_frequency=5)


In [ ]:
tokenizer = Tokenizer(model=bpe)
tokenizer.pre_tokenizer = pre_tok

In [ ]:
trainer = trainers.BpeTrainer(special_tokens\
                              =["[UNK]", "[PAD]"])

In [ ]:
data = df['review_translate'].values.tolist()

In [ ]:
tokenizer.train_from_iterator(data, trainer=trainer)

In [ ]:
tokenizer.encode('я ебав тебе в рот').tokens

In [ ]:
tokenizer.get_vocab_size()

# Save tokenizer

In [ ]:
tokenizer.save("tokenizer_30k.json")